# 1. Импорт библиотек

In [4]:
import pandas as pd
import json
import re
from sklearn.preprocessing import MultiLabelBinarizer
from datetime import datetime
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import auc, roc_auc_score, roc_curve,f1_score,mean_squared_error,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score, KFold
import ast
import matplotlib.pyplot as plt
import seaborn as sns 
from sklearn.feature_selection import f_regression, mutual_info_regression,f_classif
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,AdaBoostRegressor
from catboost import CatBoostRegressor, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
import sys
from sklearn.cluster import DBSCAN
from collections import Counter
import xgboost as xgb
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import LinearRegression

# 2.Вспомогательные функции

In [5]:
cat_list  = []
def category_list(col):
    if col not in cat_list:
        cat_list.append(col)
    print (cat_list)
    
bin_list  = []
def binary_list(col):
    if col not in bin_list:
        bin_list.append(col)
    print (bin_list)
    
num_list  = []
def numerical_list(col):
    if col not in num_list:
        num_list.append(col)
    print (num_list)
    
dum_list  = []
def dummy_list(col):
    if col not in dum_list:
        dum_list.append(col)
    print (dum_list)

def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual))

In [6]:
def pie(columns, data, title, figsize):
    fig, ax = plt.subplots(figsize=figsize, subplot_kw=dict(aspect="equal"))

    columns = columns

    data = data

    data_count = [x.split()[-1] for x in columns]


    def func(pct, allvals):
        absolute = int(np.round(pct/100.*np.sum(allvals)))
        return "{:.1f}%\n({:d})".format(pct, absolute)


    wedges, texts, autotexts = ax.pie(data, autopct=lambda pct: func(pct, data),
                                      textprops=dict(color="w"))

    ax.legend(wedges, data_count,
              title=title,
              loc="center left",
              bbox_to_anchor=(1, 0, 0.5, 1))

    plt.setp(autotexts, size=14, weight="bold")

    ax.set_title(title)

    plt.show()

def get_boxplot(column,dt):
    fig, ax = plt.subplots(figsize=(10, 10))
    sns.boxplot(x=column, y='price', data=dt, ax=ax)
    plt.xticks(rotation=45)
    ax.set_title('Boxplot for ' + column)
    plt.show()


def check_dif(col):
    for i in df[col].unique():
        if i not in df_test[col].unique():
            print('df ',i, ' not in df_test')
    for i in df_test[col].unique():
        if i not in df[col].unique():
            print('df_test ',i,' not in df')
            
def compare_cols(col):
    df_compare = pd.DataFrame(df[col].value_counts()).merge(pd.DataFrame(
        df_test[col].value_counts()), left_index=True, right_index=True)
    return df_compare
    

# 3. Загрузка датасетов

In [7]:
# без цен
df_test = pd.read_csv('../input/sf-dst-car-price-prediction/test.csv',low_memory=False, encoding = 'UTF-8')

# спарсеный с ценами 20/10/2021
df = pd.read_csv('../input/dataset-cars-autoru/dataset_V2.csv',encoding = 'UTF-8')
df = df.drop('Unnamed: 0', axis=1)
df = df.dropna()

# результат
sample_submission = pd.read_csv('../input/sf-dst-car-price-prediction/sample_submission.csv')

# 4. Сравним данные представленные в датасетах

In [8]:
print(f"Кол-во строк : {len(df_test)}, Кол-во признаков : {len(df_test.columns)}")
print(f"Кол-во строк : {len(df)}, Кол-во признаков : {len(df.columns)}")

### Посмотрим внимательнее на колонки которых нет в спарсеном датасете

In [9]:
df_test[[x for x in df_test.columns if x not in df.columns]].info()

много пропусков в complectation_dict и Владение

In [10]:
# complectation_dict
df_test.complectation_dict[2]

In [11]:
# image
df_test.image

# изображения авто, если бы мы умели определять стоимость по фото, то оставили

In [12]:
# name
df_test.name

# создадим name в df
df['name'] = df['super_gen'].apply(lambda x: ast.literal_eval(x)['human_name'])

In [13]:
# parsing_unixtime перобразем в дату
df_test['parsing_date'] = df_test['parsing_unixtime'].apply(
    lambda x: datetime.fromtimestamp(x).strftime('%Y-%m-%d'))

# создадим дату парсинга в df
df['parsing_date'] = '2021-11-12'

# parsing_unixtime удаляем

In [14]:
# priceCurrency
df_test.priceCurrency.value_counts()

# удаляем

In [15]:
# Владение
df_test.Владение[34681]

In [16]:
# sell_id
df_test.sell_id.duplicated().value_counts()
# False    34686

# создадим sell_id в df
df['sell_id'] = df['car_url'].apply(lambda x: x.split('/')[8].split('-')[0])

# проверим на дубликаты
df.sell_id.duplicated().value_counts()
# False    95114
# True       762

# удалим дубликаты
df = df.drop(df[df.duplicated(subset=['sell_id'])].index)
df = df.reset_index(drop = True)

- complectation_dict - информация практически идентичная equipment_dict => удаляем
- Владение - много не восполнимых пропусков => удаляем

In [17]:
df_test = df_test.drop(['complectation_dict','Владение','image','parsing_unixtime','priceCurrency'],axis=1)

### Посмотрим внимательнее на колонки которых нет в спарсеном датасете

In [18]:
df[[x for x in df.columns if x not in df_test.columns]].info()

In [19]:
df_test['price'] = 0

In [20]:
# сделаем последовательность колонок идентичными
df = df.reindex(columns=df_test.columns)

### Проверим идентичность Dtype

In [21]:
dtype = dict(df.dtypes == df_test.dtypes)
for i in dtype:
    if dtype[i] == False:
        print(i,'\n',
              'df',df.dtypes[i],'\n',
              'df_test',df_test.dtypes[i])

In [22]:
df.mileage = df.mileage.apply(lambda x: int(x))
df.modelDate = df.modelDate.apply(lambda x: int(x))
df.numberOfDoors = df.numberOfDoors.apply(lambda x: int(x))
df.productionDate = df.productionDate.apply(lambda x: int(x))
df.sell_id = df.sell_id.apply(lambda x: int(x))
df.price = df.price.apply(lambda x: int(x))

In [23]:
df_test['Владельцы']

In [24]:
df_test['Владельцы'] = df_test['Владельцы'].apply(lambda x: int((re.findall(r'\d', x)[0])))

Данные приведены к единому формату, рассмотрим признаки

## Обработка признаков

In [25]:
df.info()

In [26]:
# bodyType

check_dif('bodyType')
# df_test  фастбек  not in df

df_test[df_test.bodyType == 'фастбек']['model_name']
# заменим на схожий
df_test.loc[df_test[df_test['bodyType']=='фастбек'].index,'bodyType'] = 'лифтбек'

category_list('bodyType')

check_dif('bodyType')

In [27]:
# brand

check_dif('brand')
category_list('brand')

In [28]:
# color

check_dif('color')
category_list('color')

In [29]:
# engineDisplacement

check_dif('engineDisplacement')
df_test[df_test.engineDisplacement == '5.3 LTR'][[
    'brand', 'model_name', 'description']]
category_list('engineDisplacement')

# создадим engineDisplacement_num
df_test['engineDisplacement_num'] = df_test['engineDisplacement'].apply(
    lambda x: float(x.split(' ')[0]) if x.split(' ')[0] != '' else 0)
df['engineDisplacement_num'] = df['engineDisplacement'].apply(
    lambda x: float(x.split(' ')[0]) if x.split(' ')[0] != '' else 0)

numerical_list('engineDisplacement_num')

In [30]:
# enginePower

df_test['enginePower'] = df_test['enginePower'].apply(lambda x: int(x.split(sep=' ')[0]))
df['enginePower'] = df['enginePower'].apply(lambda x: int(x.split(sep=' ')[0]))

numerical_list('enginePower')

In [31]:
# equipment_dict

len(df[df.equipment_dict == '{}'])
# 29453

len(df_test[df_test.equipment_dict.isna()])
# 9996

# видим пропуски в equipment_dict, заполним их самой распространенной комплектацией в бренде, 
# для этого создадим дополнительные признаки

df_test['equipment_list'] = df_test['equipment_dict'].apply(
    lambda x: sorted(list(json.loads(x).keys())) if type(x) != float else None)

df['equipment_list'] = df['equipment_dict'].apply(lambda x: sorted(
    list(ast.literal_eval(x).keys())) if x != '{}' else None)

df['len_quip_list'] = df[df['equipment_list'].notna()]['equipment_list'].apply(lambda x: int(len(x)) )
df_test['len_quip_list'] = df_test[df_test['equipment_list'].notna()]['equipment_list'].apply(lambda x: int(len(x)) )

df['sorted_quip_list'] = df[df['equipment_list'].notna()]['equipment_list'].apply(lambda x: str(x) )
df_test['sorted_quip_list'] = df_test[df_test['equipment_list'].notna()]['equipment_list'].apply(lambda x: str(x) )

# объеденим
df_equip = df[['equipment_list', 'brand', 'len_quip_list',
               'sorted_quip_list']].append(df_test[['equipment_list', 'brand', 'len_quip_list',
               'sorted_quip_list']], sort=False).reset_index(drop=True)

# создадим словарь с распространенной комплектацией
equip_pop_dict = {}
error_dict = []
for i in df_equip[df_equip['equipment_list'].isna()]['brand'].unique():
    try:
        equip_pop_dict[i] = df_equip[(df_equip['brand'] == i) & (
            df_equip['equipment_list'].notna()) & (df_equip['len_quip_list'] > 2)]['sorted_quip_list'].value_counts().keys()[0]
    except :
        error_dict.append(i)
        continue

# заполним пропуски

df.loc[df['sorted_quip_list'].isna(),'sorted_quip_list'] = df['brand'].apply(lambda x: equip_pop_dict[x])
df_test.loc[df_test['sorted_quip_list'].isna(),'sorted_quip_list'] = df_test['brand'].apply(lambda x: equip_pop_dict[x])

df.loc[df['equipment_list'].isna(),'equipment_list'] = df['sorted_quip_list'].apply(lambda x: x[2:-2].split("', '"))
df_test.loc[df_test['equipment_list'].isna(),'equipment_list'] = df_test['sorted_quip_list'].apply(lambda x: x[2:-2].split("', '"))

# удалим вспомогательные признаки и обновим len_quip_list

df = df.drop(['equipment_dict','sorted_quip_list'],axis=1)
df_test = df_test.drop(['equipment_dict','sorted_quip_list'],axis=1)

df['len_quip_list'] = df['equipment_list'].apply(lambda x: int(len(x)))
df_test['len_quip_list'] = df_test['equipment_list'].apply(lambda x: int(len(x)))

# добавим в dummy_list, потом извлечем признаки перед обработкой
dummy_list('equipment_list')
numerical_list('len_quip_list')

In [32]:
# fuelType

check_dif('fuelType')
# разницы не обнаружено

category_list('fuelType')

In [33]:
# mileage

df_test['mileage']

numerical_list('mileage')

In [34]:
# modelDate

check_dif('modelDate')
category_list('modelDate')

In [35]:
# model_info

#извлечем признак morfology
df['morfology'] = df['model_info'].apply(
    lambda x: 'FEMININE' if len(ast.literal_eval(x)) == 1 else 'MASCULINE')
df_test['morfology'] = df_test['model_info'].apply(
    lambda x: 'FEMININE' if len(json.loads(x)['morphology']) == 1 else 'MASCULINE')

binary_list('morfology')

df = df.drop(['model_info'],axis=1)
df_test = df_test.drop(['model_info'],axis=1)

In [36]:
# model_name

df['model_name'] = df['car_url'].apply(lambda x: x.split('/')[7].upper())
check_dif('model_name')

# избавимся от моделей которых нет в df_test
for i in df['model_name'].unique():
    if i not in df_test['model_name'].unique():
        df = df.drop(df[df['model_name'] == i].index).reset_index(drop=True)


In [37]:
category_list('model_name')

In [38]:
# name

df.name
df_test.name
# признак состоит из нескольких признаков присутствующих в датасетах, удалим его
df = df.drop('name',axis=1)
df_test = df_test.drop('name',axis=1)

In [39]:
# numberOfDoors

check_dif('numberOfDoors')
category_list('numberOfDoors')

In [40]:
# productionDate

check_dif('productionDate')
# датасете с ценами отсутствуют раритетные модели, разберемся с этим позже
numerical_list('productionDate')

In [41]:
# super_gen

df.super_gen[0]
df_test.super_gen[0]

# '{"id":"10373605","displacement":1197,"engine_type":"GASOLINE","gear_type":"FORWARD_CONTROL",
# "transmission":"ROBOT","power":105,"power_kvt":77,"human_name":"1.2 AMT (105 л.с.)","acceleration":10.5,
# "clearance_min":155,"fuel_rate":5}'

# извлечем признаки acceleration, clearance_min, fuel_rate

df['acceleration_super_gen'] = df['super_gen'].apply(
    lambda x: ast.literal_eval(x)['acceleration'] if 'acceleration' in ast.literal_eval(x).keys() else -1)

df['clearance_min_super_gen'] = df['super_gen'].apply(
    lambda x: ast.literal_eval(x)['clearance_min'] if 'clearance_min' in ast.literal_eval(x).keys() else -1)

df['fuel_rate_super_gen'] = df['super_gen'].apply(
    lambda x: ast.literal_eval(x)['fuel_rate'] if 'fuel_rate' in ast.literal_eval(x).keys() else -1)

df_test['acceleration_super_gen'] = df_test['super_gen'].apply(
    lambda x: json.loads(x)['acceleration'] if 'acceleration' in json.loads(x).keys() else -1)

df_test['clearance_min_super_gen'] = df_test['super_gen'].apply(
    lambda x: json.loads(x)['clearance_min'] if 'clearance_min' in json.loads(x).keys() else -1)

df_test['fuel_rate_super_gen'] = df_test['super_gen'].apply(
    lambda x: json.loads(x)['fuel_rate'] if 'fuel_rate' in json.loads(x).keys() else -1)

df = df.drop('super_gen', axis=1)
df_test = df_test.drop('super_gen', axis=1)

In [42]:
df.fuel_rate_super_gen.value_counts()

In [43]:
# acceleration_super_gen

df['acceleration_super_gen'].value_counts()

# получилось много пропущеных строк в acceleration_super_gen,fuel_rate_super_gen удалим их
df = df.drop(['acceleration_super_gen','fuel_rate_super_gen'], axis=1)
df_test = df_test.drop(['acceleration_super_gen','fuel_rate_super_gen'], axis=1)
numerical_list('clearance_min_super_gen')


In [44]:
# vehicleConfiguration

check_dif('vehicleConfiguration')
df.vehicleConfiguration.value_counts()

# нет полезной информации, удаляем
df = df.drop('vehicleConfiguration', axis=1)
df_test = df_test.drop('vehicleConfiguration', axis=1)


In [45]:
# vehicleTransmission

check_dif('vehicleTransmission')
category_list('vehicleTransmission')

In [46]:
# vendor

check_dif('vendor')

df = df.drop(df[df['vendor'] == 'AMERICAN'].index).reset_index(drop=True)
binary_list('vendor')

In [47]:
# Владельцы

check_dif('Владельцы')

df['Владельцы'] = df['Владельцы'].apply(lambda x: 3 if x == 4 else x)
category_list('Владельцы')

In [48]:
# ПТС

check_dif('ПТС')

df_test['ПТС'] = df_test['ПТС'].apply(
    lambda x: 'ORIGINAL' if x == 'Оригинал' else 'DUPLICATE' if x == 'Дубликат' else 'DUPLICATE')
binary_list('ПТС')

In [49]:
check_dif('ПТС')

In [50]:
# Привод

check_dif('Привод')

df_test['Привод'] = df_test['Привод'].apply(
    lambda x: 'FORWARD_CONTROL' if x == 'передний' else 'ALL_WHEEL_DRIVE' if x == 'полный' else 'REAR_DRIVE')

category_list('Привод')

In [51]:
# Руль

check_dif('Руль')

df_test['Руль'] = df_test['Руль'].apply(
    lambda x: 'LEFT' if x == 'Левый' else 'RIGHT')

binary_list('Руль')

In [52]:
# Состояние

df.Состояние.value_counts()
# True    94549

# все данные идентичны, удаляем
df = df.drop('Состояние', axis =1)
df_test = df_test.drop('Состояние', axis =1)

In [53]:
# Таможня

df.Таможня.value_counts()
# True    94549

# все данные идентичны, удаляем
df = df.drop('Таможня', axis =1)
df_test = df_test.drop('Таможня', axis =1)

In [54]:
# parsing_date

check_dif('parsing_date')

# извлечем год, месяц
df['parsing_year'] = 2021
df['parsing_month'] = 11
df['parsing_day'] = 12

df_test['parsing_year'] = df_test['parsing_date'].apply(lambda x: int(x.split('-')[0]))
df_test['parsing_month'] = df_test['parsing_date'].apply(lambda x: int(x.split('-')[1]))
df_test['parsing_day'] = df_test['parsing_date'].apply(lambda x: int(x.split('-')[2]))

# Добавим признаки возраст авто и возраст модели
df['model_age_month'] =  (df.parsing_year - df.modelDate)*12 + df.parsing_month
df_test['model_age_month'] =  (df_test.parsing_year - df_test.modelDate)*12 + df_test.parsing_month

df['car_age_month'] =  (df.parsing_year - df.productionDate)*12 + df.parsing_month
df_test['car_age_month'] =  (df_test.parsing_year - df_test.productionDate)*12 + df_test.parsing_month

# удалим не нужные признаки
df = df.drop(['parsing_year','parsing_month','parsing_day', 'parsing_date'],axis=1)
df_test = df_test.drop(['parsing_year','parsing_month','parsing_day','parsing_date'],axis=1)

numerical_list('model_age_month')
numerical_list('car_age_month')

In [55]:
# удалим оставшиеся не нужные признаки

df = df.drop(['car_url','description'],axis=1)
df_test = df_test.drop(['car_url','description'],axis=1)

In [56]:
# проверим идентичность признаков в обоих датасетах
len(cat_list)+len(bin_list)+len(num_list)+len(dum_list) == len(df.drop(['sell_id','price'],axis=1).columns)


<h1> Итог:</h1> Мы сделали первичную обработку признаков, избавились от неидентичных данных и привели оба датасета к единому формату. В процессе мы выявили, что в данных с ценой нет некоторых моделей для предсказания, попробуем решить этот вопрос при анализе данных.

# 5. EDA, Feature engineering

## bodyType

In [57]:
# Попробуем сравнить цены по группам
df['bodyType_group'] = df['bodyType'].apply(lambda x: re.findall(r'(\D+)', x)[0])
df['bodyType_group'] = df['bodyType_group'].apply(lambda x: x.split(' ')[0])
df['bodyType_group'] = df['bodyType_group'].apply(lambda x: x.split('-')[0])

df_test['bodyType_group'] = df_test['bodyType'].apply(lambda x: re.findall(r'(\D+)', x)[0])
df_test['bodyType_group'] = df_test['bodyType_group'].apply(lambda x: x.split(' ')[0])
df_test['bodyType_group'] = df_test['bodyType_group'].apply(lambda x: x.split('-')[0])

df['bodyType_group'].value_counts()
get_boxplot('bodyType_group',df)
for i in df['bodyType_group'].value_counts().keys()[:5]:
    get_boxplot('bodyType',df[df['bodyType_group'] == i])
    
category_list('bodyType_group')

Видим много выбросов, скорее всего сказываются цены на премиальные марки, в дальнейшем сгруппируем по ценовому сегменту

In [58]:
# Создадим сегменты бранда
df['segment_brand'] = df['brand'].apply(lambda x: 'PREMIUM' if x in [
    'AUDI','VOLVO','BMW','INFINITI','MERCEDES','LEXUS'] else 'MASS_BRAND')
df_test['segment_brand'] = df_test['brand'].apply(lambda x: 'PREMIUM' if x in [
    'AUDI','VOLVO','BMW','INFINITI','MERCEDES','LEXUS'] else 'MASS_BRAND')
    
binary_list('segment_brand')

In [59]:
get_boxplot('segment_brand',df)

Предположение стоимости бренда подтвердилась. Посмотрим на среднюю стоимость по каждому классу бренда.

In [60]:
print(df[df['segment_brand'] == 'PREMIUM'].groupby(
    'bodyType_group')['price'].median().sort_values(ascending=False))

print(df[df['segment_brand'] == 'MASS_BRAND'].groupby(
    'bodyType_group')['price'].median().sort_values(ascending=False))

# Проранжируем по медианной стоимости

body_range = list(df[df['segment_brand'] == 'PREMIUM'].groupby(
    'bodyType_group')['price'].median().sort_values(ascending=False).index)

df.loc[df[df['segment_brand'] == 'PREMIUM'].index,'body_range'] = df[
    df['segment_brand'] == 'PREMIUM']['bodyType_group'].apply(lambda x: body_range.index(x))

df_test.loc[df_test[df_test['segment_brand'] == 'PREMIUM'].index,'body_range'] = df_test[
    df_test['segment_brand'] == 'PREMIUM']['bodyType_group'].apply(lambda x: body_range.index(x))

body_range = list(df[df['segment_brand'] == 'MASS_BRAND'].groupby(
    'bodyType_group')['price'].median().sort_values(ascending=False).index)

df.loc[df[df['segment_brand'] == 'MASS_BRAND'].index,'body_range'] = df[
    df['segment_brand'] == 'MASS_BRAND']['bodyType_group'].apply(lambda x: body_range.index(x))

df_test.loc[df_test[df_test['segment_brand'] == 'MASS_BRAND'].index,'body_range'] = df_test[
    df_test['segment_brand'] == 'MASS_BRAND']['bodyType_group'].apply(lambda x: body_range.index(x))

category_list('body_range')

<h3>Итог по bodyType:</h3> немного обобщили группы, сегментировали по бренду, проранжировали по медианной стоимости

## brand

In [61]:
plt.subplot(1, 2, 1) 
df['brand'].value_counts().plot(kind='bar')
plt.subplot(1, 2, 2)
df_test['brand'].value_counts().plot(kind='bar',figsize=(10, 5))
category_list('brand')

In [62]:
get_boxplot('brand',df)

Распределение в датасетах по брендам не равномерное, будем предсказывать цену предварительно поделив выборку на бренды,выбросы по цене в брендах, на это может влиять год производства, комплектация, пробег, модель авто

## color

In [63]:
# посмотрим на медианную стоимость цвета по классу
df[df['segment_brand'] == 'PREMIUM'].groupby(
    'color')['price'].median().sort_values(ascending=False)

df[df['segment_brand'] == 'MASS_BRAND'].groupby(
    'color')['price'].median().sort_values(ascending=False)

# Проранжируем по медианной стоимости
color_range = list(df[df['segment_brand'] == 'PREMIUM'].groupby(
    'color')['price'].median().sort_values(ascending=False).index)

df.loc[df[df['segment_brand'] == 'PREMIUM'].index,'color_range'] = df[
    df['segment_brand'] == 'PREMIUM']['color'].apply(lambda x: color_range.index(x))

df_test.loc[df_test[df_test['segment_brand'] == 'PREMIUM'].index,'color_range'] = df_test[
    df_test['segment_brand'] == 'PREMIUM']['color'].apply(lambda x: color_range.index(x))

color_range = list(df[df['segment_brand'] == 'MASS_BRAND'].groupby(
    'color')['price'].median().sort_values(ascending=False).index)

df.loc[df[df['segment_brand'] == 'MASS_BRAND'].index,'color_range'] = df[
    df['segment_brand'] == 'MASS_BRAND']['color'].apply(lambda x: color_range.index(x))

df_test.loc[df_test[df_test['segment_brand'] == 'MASS_BRAND'].index,'color_range'] = df_test[
    df_test['segment_brand'] == 'MASS_BRAND']['color'].apply(lambda x: color_range.index(x))

category_list('color_range')

In [64]:
df[df['segment_brand'] == 'MASS_BRAND'].groupby(
    'color')['price'].median().sort_values(ascending=False)

In [65]:
df[df['segment_brand'] == 'PREMIUM'].color.value_counts()

In [66]:
df[df['segment_brand'] == 'MASS_BRAND'].color.value_counts()

<h3>Итог по color:</h3> проранжировали по медианной стоимости, интересно, что популярность цвета отличается от стоимости бренда

## engineDisplacement, engineDisplacement_num

Сгрупируем по объему: микролитражные (до 1,1 л), малолитражные (1,2-1,5 л), среднелитражные (1,6-3,5 л) и крупнолитражные (свыше 3,5 л)

In [67]:
df['engineDisplacement_group'] = df['engineDisplacement_num'].apply(lambda x: 'микролитражные' if 0 < x <= 1.1
                                                                    else 'электро' if x == 0
                                                                    else 'малолитражные' if 1.1 < x <= 1.5 else 'среднелитражные' if 1.5 < x <= 3.5
                                                                    else 'крупнолитражные')

df_test['engineDisplacement_group'] = df_test['engineDisplacement_num'].apply(lambda x: 'микролитражные' if 0 < x <= 1.1
                                                                              else 'электро' if x == 0
                                                                              else 'малолитражные' if 1.1 < x <= 1.5
                                                                              else 'среднелитражные' if 1.5 < x <= 3.5
                                                                              else 'крупнолитражные')
category_list('engineDisplacement_group')

In [68]:
compare_cols('engineDisplacement_group')

In [69]:
df.groupby('engineDisplacement_group')['price'].median().sort_values(ascending=False)

<h3>Итог по engineDisplacement:</h3> сгрупировали по объему двигателя, выше объем больше стоимость, интересно что электрокары оказались по стоимости между малолитражными и среднелитражными

## enginePower

In [70]:
df.enginePower.hist()

In [71]:
sns.pairplot(df[['enginePower','price']])

In [72]:
# сгруппируем, воспользовавшись стандартным отклонением
IQR = df.enginePower.quantile(
    0.75) - df.enginePower.quantile(0.25)
perc25 = df.enginePower.quantile(0.25)
perc75 = df.enginePower.quantile(0.75)
print('25th percentile: {},'.format(perc25), '75th percentile: {},'.format(perc75),
      "IQR: {}, ".format(IQR), "Пределы: [{f}, {l}].".format(f=perc25 - 1.5*IQR, l=perc75 + 1.5*IQR))

# Стандартное отклонение
s = np.std(df.enginePower, ddof=1)
m = df.enginePower.mean()

In [73]:
df['enginePower_group'] = df.enginePower.apply(lambda x: 'low' if x <= 118 else 'medium' if 118 < x < 220
                                               else 'hight' if 220 <= x < 373 else 'extra_hight')
df_test['enginePower_group'] = df_test.enginePower.apply(lambda x: 'low' if x <= 118 else 'medium' if 118 < x < 220
                                               else 'hight' if 220 <= x < 373 else 'extra_hight')
category_list('enginePower_group')

In [74]:
get_boxplot('enginePower_group',df)

In [75]:
# посмотрим на модели дадим и название LUXE
print(df.query('enginePower > 373')['model_name'].value_counts())
print(df_test.query('enginePower > 373')['model_name'].value_counts())

for i in df.query('enginePower > 373').index:
    df.loc[i, 'segment_brand'] = 'LUXE'
    
for i in df_test.query('enginePower > 373').index:
    df_test.loc[i, 'segment_brand'] = 'LUXE'

In [76]:
get_boxplot('segment_brand',df)

<h3>Итог по enginePower:</h3> есть небольшая корреляция с ценой, создали групы на основании мощности двигателя, добавили новый сегмент в segment_brand

## fuelType

In [77]:
compare_cols('fuelType')

In [78]:
df.groupby('fuelType')['price'].median().sort_values(ascending=False)

## mileage

In [79]:
sns.pairplot(df[['mileage','price']])

In [80]:
df.groupby('brand')['mileage'].mean().sort_values(ascending=False)

In [81]:
df_test.groupby('brand')['mileage'].mean().sort_values(ascending=False)

Важный признак, используем его для создания новых

## modelDate

In [82]:
df.modelDate.hist()

In [83]:
sns.pairplot(df[['modelDate','price']])

In [84]:
# разделим на категории основываясь по дате модели
df['modelDate_group'] = df['modelDate'].apply(lambda x: 'раритет' if x <= 1970 else 'старейшие' if 1970 < x < 2000
                                              else 'старые' if 2000 <= x < 2010
                                              else 'новые' if 2010 <= x < 2015
                                              else 'новейшие')
df_test['modelDate_group'] = df['modelDate'].apply(lambda x: 'раритет' if x <= 1970 else 'старейшие' if 1970 < x < 2000
                                                   else 'старые' if 2000 <= x < 2010
                                                   else 'новые' if 2010 <= x < 2015
                                                   else 'новейшие')


category_list('modelDate_group')

In [85]:
get_boxplot('modelDate_group',df)

In [86]:
compare_cols('modelDate_group')

In [87]:
# Посмотрим на productionDate и modelDate

df['fresh_model'] = df.productionDate - df.modelDate
df_test['fresh_model'] = df_test.productionDate - df_test.modelDate

numerical_list('fresh_model')


## model_name

In [88]:
df.groupby('model_name')['price'].max().sort_values(ascending=False)

## numberOfDoors

In [89]:
df.groupby('numberOfDoors')['price'].median().sort_values(ascending=False)

## productionDate

In [90]:
# Создадим признак пробег в месяц
df['mileage_age'] = df.mileage/df.car_age_month
df['mileage_owners'] = df.mileage/df.Владельцы

df_test['mileage_age'] = df.mileage/df.car_age_month
df_test['mileage_owners'] = df.mileage/df.Владельцы


numerical_list('mileage_age')
numerical_list('mileage_owners')

## vehicleTransmission

In [91]:
compare_cols('vehicleTransmission')

In [92]:
df.groupby('vehicleTransmission')['price'].median().sort_values(ascending=False)

## vendor

In [93]:
compare_cols('vendor')

## Владельцы

In [94]:
df.groupby('Владельцы')['price'].median().sort_values(ascending=False)

## ПТС

In [95]:
df.groupby('ПТС')['price'].median().sort_values(ascending=False)

## Привод

In [96]:
df.groupby('Привод')['price'].median().sort_values(ascending=False)

In [97]:
get_boxplot('Привод',df)

Полноприводные модели дороже, задний привод в целом дешевле, но сказывается стоимость люкса

## Руль

In [98]:
df.groupby('Руль')['price'].median().sort_values(ascending=False)

In [99]:
len(cat_list)+len(bin_list)+len(num_list)+len(dum_list) == len(df.drop(['sell_id','price'],axis=1).columns)


## Price

In [100]:
# Посмотрим на распределение

df.price.hist(bins=100)

In [101]:
# распределение не равномерное для обучения будем использовать логарифмирование
np.log(df.price).hist(bins=100)

### т.к. датасет для предсказания цен был на октябрь 2020 года, а парсинг на ноябрь 2021,  цены на авто изменились, внесем поправку согласно таблицы (https://mag.auto.ru/article/risepricescalltouchpres/)
<img src="https://autoru-mag.s3.yandex.net/2021/04/19/aeece79bea854e7aa442f854295e9181.png" width="500" height="600">

In [102]:
price_change = {
    'MERCEDES': 0.211,
    'SKODA': 0.2,
    'AUDI': 0.228,
    'HONDA': 0.167,
    'VOLVO': 0.152,
    'BMW': 0.242,
    'NISSAN': 0.191,
    'INFINITI': 0.2,
    'MERCEDES': 0.211,
    'TOYOTA': 0.2,
    'LEXUS': 0.2,
    'VOLKSWAGEN': 0.25,
    'MITSUBISHI':0.2
}
for i in df.brand.unique():
    df.loc[df[df.brand == i].index, 'price'] = df.loc[df[df.brand == i].index, 'price'].apply(lambda x: x - x * price_change[i] )


## Корреляция признаков

In [103]:
fig, ax = plt.subplots(figsize=(15,15)) 
sns.heatmap(df.corr(), vmin=-1, vmax=1, square=True,
              annot=True, fmt=".3f", linewidths=1,ax=ax)

In [104]:
df_corr = pd.DataFrame(df.corr())
help_list=[]
for i in df_corr:
    for n in df_corr[i]:
        if df_corr.index[df_corr[i]== n][0] != i:
            if n> 0.6 or n<-0.6:
                if n not in help_list:
                    help_list.append(n)
                    print('Row/Col :','\n',df_corr.index[df_corr[i]== n][0],'/',i,'\n', n,'\n')

In [105]:
df_corr = pd.DataFrame(df_test.drop('price',axis=1).corr())
help_list=[]
for i in df_corr:
    for n in df_corr[i]:
        if df_corr.index[df_corr[i]== n][0] != i:
            if n> 0.6 or n<-0.6:
                if n not in help_list:
                    help_list.append(n)
                    print('Row/Col :','\n',df_corr.index[df_corr[i]== n][0],'/',i,'\n', n,'\n')

In [106]:
# удалим признаки с сильной корреляцией

df = df.drop(['productionDate','modelDate'],axis=1)
df_test = df_test.drop(['productionDate','modelDate'],axis=1)

num_list.remove('productionDate')
cat_list.remove('modelDate')

In [107]:
len(cat_list)+len(bin_list)+len(num_list)+len(dum_list) == len(df.drop(['sell_id','price'],axis=1).columns)

In [108]:
dtype = dict(df.dtypes == df_test.dtypes)
for i in dtype:
    if dtype[i] == False:
        print(i,'\n',
              'df',df.dtypes[i],'\n',
              'df_test',df_test.dtypes[i])

In [109]:
df['Владельцы'] = df['Владельцы'].apply(lambda x: int(x))

## Объеденим данные

In [110]:
data = df_test.append(df, sort=False).reset_index(drop=True)

In [111]:
data.select_dtypes(include='float').columns

In [112]:
for i in ['body_range', 'color_range']:
    data[i] = data[i].apply(lambda x: int(x))

In [113]:
X = data.query('price != 0').drop(['price','sell_id'],axis=1)
X.shape

In [114]:
X_val = data.query('price == 0').drop(['price','sell_id'],axis=1)
X_val.shape

In [171]:
y = data.query('price != 0')['price']
y.shape

# 6. Machine Learning

## CatBoost Regressor

### Подготовим к обучению используя конструктор Pool для CatBoost

In [116]:
# # подготовим признаки из dum_list
# 
# mlb = MultiLabelBinarizer()
# 
# # equipment_dict
# 
# X_dumm_equipment_dict = pd.DataFrame(mlb.fit_transform(
#     X['equipment_list']), columns=mlb.classes_, index=df.index)
# 
# X_val_dumm_equipment_dict = pd.DataFrame(mlb.fit_transform(
#     X_val['equipment_list']), columns=mlb.classes_, index=df_test.index)
# 
# # приведем в соответсвие equipment_dict в df и df_test, удалим колонки которых нет в df и df_test соответственно 
# 
# delete_cols_df = []
# for i in X_dumm_equipment_dict.columns:
#     if i not in X_val_dumm_equipment_dict.columns:
#         delete_cols_df.append(i)
#         
# delete_cols_df_test = []
# for i in X_val_dumm_equipment_dict.columns:
#     if i not in X_dumm_equipment_dict.columns:
#         delete_cols_df_test.append(i)
# 
# dumm_equipment_dict = X_val_dumm_equipment_dict.drop(delete_cols_df_test, axis=1).append(
#     X_dumm_equipment_dict.drop(delete_cols_df, axis=1), sort=False).reset_index(drop=True)
# 
# 
# X_CatB_dumm = dumm_equipment_dict
# X_CatB_dumm.shape

In [117]:
# X_all_for_CB = X.drop('equipment_list', axis=1).merge(
#     X_CatB_dumm.loc[data.query('price != 0').index], left_index=True, right_index=True)
# X_all_for_CB_val = X_val.drop('equipment_list', axis=1).merge(
#     X_CatB_dumm.loc[data.query('price == 0').index], left_index=True, right_index=True)
# 
# pool_all = Pool(X_all_for_CB, np.log(y), feature_names=list(X_all_for_CB.columns),
#                 cat_features=list(X_all_for_CB.select_dtypes(include='object').columns))

In [118]:
# skf = KFold(n_splits=4, random_state=42, shuffle=True)
# cat_boost_mape = []
# 
# cat_boost = CatBoostRegressor(iterations=5000,
#                               random_seed=42,
#                               learning_rate=0.5,
#                               eval_metric='MAPE',
#                               od_type='Iter',
#                               od_wait=200,
#                               silent=True)
# 
# for train_index, test_index in skf.split(X_all_for_CB, y):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X_all_for_CB[X.index.isin(train_index)], X_all_for_CB[X.index.isin(test_index)]
#     y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
#   
#     cat_boost.fit(
#         X = pool_all,
#         eval_set=(X_test, np.log(y_test)),
#         verbose_eval=0,
#         use_best_model=True,
#         plot=True)
# 
# 
#     y_pred = np.exp(cat_boost.predict(X_test))
# 
#     print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
#     
#     cat_boost_mape.append(mape(y_test, y_pred))

In [119]:
# y_pred = np.exp(cat_boost.predict(X_all_for_CB_val))
# # округлим итоговую цену
# y_pred_round = np.round(y_pred/100000,2)*100000
# sample_submission['price'] = y_pred_round
# sample_submission.to_csv('submission_1.csv', index=False)

Точность модели по метрике MAPE: 8.33%<br>
submission 34.20671% # Pool, K-fold

### Подготовим модель используя в качестве признаков числовой массив

In [120]:
# # подготовим признаки из dum_list
# 
# mlb = MultiLabelBinarizer()
# 
# # equipment_dict
# 
# X_dumm_equipment_dict = pd.DataFrame(mlb.fit_transform(
#     X['equipment_list']), columns=mlb.classes_, index=df.index)
# 
# X_val_dumm_equipment_dict = pd.DataFrame(mlb.fit_transform(
#     X_val['equipment_list']), columns=mlb.classes_, index=df_test.index)
# 
# # приведем в соответсвие equipment_dict в df и df_test, удалим колонки которых нет в df и df_test соответственно 
# 
# delete_cols_df = []
# for i in X_dumm_equipment_dict.columns:
#     if i not in X_val_dumm_equipment_dict.columns:
#         delete_cols_df.append(i)
#         
# delete_cols_df_test = []
# for i in X_val_dumm_equipment_dict.columns:
#     if i not in X_dumm_equipment_dict.columns:
#         delete_cols_df_test.append(i)
# 
# dumm_equipment_dict = X_val_dumm_equipment_dict.drop(delete_cols_df_test, axis=1).append(
#     X_dumm_equipment_dict.drop(delete_cols_df, axis=1), sort=False).reset_index(drop=True)
# 
# 
# X_CatB_dumm = dumm_equipment_dict
# X_CatB_dumm.shape
# X_rf_cat_bin = data[bin_list].copy()
# 
# le = LabelEncoder()
# for i in X_rf_cat_bin:
#     X_rf_cat_bin.loc[X_rf_cat_bin.index,i] = le.fit_transform(X_rf_cat_bin.loc[X_rf_cat_bin.index,i])
#     
# X_rf_cat_bin = X_rf_cat_bin.values
#     
# print('X_rf_cat_bin ',X_rf_cat_bin.shape)
# 
# # числовые признаки
# 
# sc = StandardScaler()
# 
# X_rf_cat_num = data.select_dtypes(include=['int','float']).drop(['sell_id','price'],axis=1)
# X_rf_cat_num = sc.fit_transform(X_rf_cat_num.values)
# 
# print('X_rf_cat_num ',X_rf_cat_num.shape)
# 
# # dummy 
# 
# X_rf_cat_dumm = X_CatB_dumm.values
# 
# print('X_rf_cat_dumm ',X_rf_cat_dumm.shape)
# 
# # Категориальные
# 
# X_rf_cat = data.select_dtypes(include='object')
# X_rf_cat
# 
# X_rf_cat = OneHotEncoder(sparse = False).fit_transform(X_rf_cat.drop('equipment_list',axis=1).values)
# 
# print('X_rf_cat ',X_rf_cat.shape)
# 
# # объеденим
# 
# X_rf_full = np.hstack([X_rf_cat_bin,X_rf_cat_num,X_rf_cat_dumm,X_rf_cat])
# 
# print('X_rf_full ',X_rf_full.shape)
# 
# # разделим на тест и валидацию
# 
# X_rf_test = X_rf_full[data.query('price != 0').index]
# X_rf_val = X_rf_full[data.query('price == 0').index]
# 
# print('X_rf_test ',X_rf_test.shape)
# print('X_rf_val ',X_rf_val.shape)

In [121]:
# skf = KFold(n_splits=5, random_state=42, shuffle=True)
# cat_boost_mape = []
# 
# cat_boost = CatBoostRegressor(iterations=5000,
#                               random_seed=42,
#                               learning_rate=0.1,
#                               eval_metric='MAPE',
#                               od_type='Iter',
#                               od_wait=200,
#                               silent=True)
# 
# for train_index, test_index in skf.split(X_rf_test, y.values):
#     print("TRAIN:", train_index, "TEST:", test_index)
#     X_train, X_test = X_rf_test[X.index.isin(train_index)], X_rf_test[X.index.isin(test_index)]
#     y_train, y_test = y[y.index.isin(train_index)], y[y.index.isin(test_index)]
#   
#     cat_boost.fit(X_train, np.log(y_train),
#        eval_set=(X_test, np.log(y_test)),
#        verbose_eval=0,
#        use_best_model=True,
#                   plot=True
#        )
# 
# 
#     y_pred = np.exp(cat_boost.predict(X_test))
# 
#     print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
#     
#     cat_boost_mape.append(mape(y_test, y_pred))

In [122]:
# y_pred = np.exp(cat_boost.predict(X_rf_val))
# sample_submission['price'] = y_pred
# sample_submission.to_csv('submission_1.csv', index=False)

Точность модели по метрике MAPE: 13.61%<br>
submission 22.24608% # без pool<br>

Используя массив в качестве подачи данных в модель результат на тесте ухудшился, но на сабмите показал себя лучше. Разница в оценке может говорить о переобучении, хотя мы использовали k fold разбиение.
Попробуем подобрать признаки используя select_features



### Подберем категориальные признаки

In [129]:
# cat_col = list(X.select_dtypes(include='object').columns)
# cat_col.remove('equipment_list') # удалим колонку для dumm

# # разделим выборку на train и test
# X_train, X_test, y_train, y_test = train_test_split(X[cat_col], np.log(y), test_size=0.30, random_state=42)

# # подготовим категориальные признаки
# features_for_select = cat_col
# cat_features = cat_col
# feature_names =  cat_col

# подготовим Pool
# train_pool = Pool(X_train, y_train, feature_names = feature_names, cat_features = cat_features)
# test_pool = Pool(X_test, y_test, feature_names = feature_names,cat_features = cat_features)

# модель на 5 000 итераций learning_rate = 0.1
# model = CatBoostRegressor(iterations=10000, 
#                               random_seed=42,
#                               learning_rate = 0.1,
#                               eval_metric='MAPE',
#                               od_type = 'Iter',
#                               od_wait = 200,
#                           rsm = 0.1,
#                               silent=True)

# # выбор категориальных признаков из 13 выберем 5 важных
# summary = model.select_features(
#     train_pool,
#     eval_set=test_pool,
#     features_for_select= features_for_select,
#     num_features_to_select=7,
#     steps=10,
#     algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
#     shap_calc_type=EShapCalcType.Regular,
#     train_final_model=True,
#     logging_level='Silent',
#     plot=True
# )

In [130]:
# summary['selected_features_names']

In [175]:
summary_cat = ['brand',
 'engineDisplacement',
 'fuelType',
 'model_name',
 'bodyType_group',
 'enginePower_group',
 'modelDate_group']

# ### Подберем числовые признаки

In [132]:
# num_col = list(X.select_dtypes(include='int').columns) + list(X.select_dtypes(include='float').columns)

# X_num_CatB_SS = StandardScaler().fit_transform(X[num_col].values)

# # разделим выборку на train и test
# X_train, X_test, y_train, y_test = train_test_split(X_num_CatB_SS, np.log(y), test_size=0.30, random_state=42)

# # подготовим категориальные признаки
# features_for_select = num_col
# feature_names =  num_col

# подготовим Pool
# train_pool = Pool(X_train, y_train, feature_names = feature_names)
# test_pool = Pool(X_test, y_test, feature_names = feature_names)

# # модель на 10 000 итераций learning_rate = 0.1
# model = CatBoostRegressor(iterations=10000, 
#                               random_seed=42,
#                               learning_rate = 0.1,
#                               eval_metric='MAPE',
#                               od_type = 'Iter',
#                               od_wait = 200,
#                           rsm = 0.1,
#                               silent=True)
# 
# # выбор категориальных признаков из 13 выберем 5 важных
# summary_num = model.select_features(
#     train_pool,
#     eval_set=test_pool,
#     features_for_select= features_for_select,
#     num_features_to_select=6,
#     steps=10,
#     algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
#     shap_calc_type=EShapCalcType.Regular,
#     train_final_model=True,
#     logging_level='Silent',
#     plot=True
# )

In [133]:
# summary_num['selected_features_names']

In [174]:
summary_num = ['enginePower',
 'mileage',
 'model_age_month',
 'car_age_month',
 'body_range',
 'engineDisplacement_num']

### Подготовим модель на выбранных признаках

In [176]:
# числовые признаки
sc = StandardScaler()

X_CatB_num = sc.fit_transform(data[summary_num])
X_CatB_num = pd.DataFrame(X_CatB_num, columns = summary_num)
X_CatB_num

In [177]:
# подготовим признаки из dum_list

mlb = MultiLabelBinarizer()

# equipment_dict

X_dumm_equipment_dict = pd.DataFrame(mlb.fit_transform(
    X['equipment_list']), columns=mlb.classes_, index=df.index)

X_val_dumm_equipment_dict = pd.DataFrame(mlb.fit_transform(
    X_val['equipment_list']), columns=mlb.classes_, index=df_test.index)

# приведем в соответсвие equipment_dict в df и df_test, удалим колонки которых нет в df и df_test соответственно 

delete_cols_df = []
for i in X_dumm_equipment_dict.columns:
    if i not in X_val_dumm_equipment_dict.columns:
        delete_cols_df.append(i)
        
delete_cols_df_test = []
for i in X_val_dumm_equipment_dict.columns:
    if i not in X_dumm_equipment_dict.columns:
        delete_cols_df_test.append(i)

dumm_equipment_dict = X_val_dumm_equipment_dict.drop(delete_cols_df_test, axis=1).append(
    X_dumm_equipment_dict.drop(delete_cols_df, axis=1), sort=False).reset_index(drop=True)


X_CatB_dumm = dumm_equipment_dict
X_CatB_dumm.shape

In [178]:
# категориальные признаки

X_CatB_category = data[summary_cat]
X_CatB_category.shape

In [179]:
# Объеденим все признаки

X_CatB = X_CatB_category
for i in [X_CatB_dumm,X_CatB_num]:
    X_CatB = X_CatB.merge(i, left_index=True, right_index=True)

X_CatB.shape

In [180]:
# разделим на тест и валидацию
X_CatB_test = X_CatB.loc[data.query('price != 0').index]
X_CatB_val = X_CatB.loc[data.query('price == 0').index]

In [181]:
# подготовим модель, разобьем на три фолда

valid_frac = 0.15
test_frac = 0.15

valid_count = int(valid_frac * X.shape[0])
test_count = int(test_frac * X.shape[0])

X_train, X_valid_test, y_train, y_valid_test = train_test_split(X_CatB_test, y,
    test_size=(valid_count + test_count), random_state=42)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_valid_test, y_valid_test, test_size=test_count, random_state=42)

In [182]:
# готовим pool

feature_names = list(X_CatB_test.columns)
cat_features = summary_cat

pool = Pool(X_CatB_test, np.log(y), feature_names = feature_names, cat_features = cat_features)

In [ ]:
# модель
cat_boost = CatBoostRegressor(iterations=10000, 
                              random_seed=42,
                              learning_rate = 0.1,
                              eval_metric='MAPE',
                              od_type = 'Iter',
                              od_wait = 200,
                              silent=True)
cat_boost.fit(
    X = pool, 
    eval_set=(X_test, np.log(y_test)),
    verbose_eval=0, 
    use_best_model=True, 
    plot=True)

In [143]:
y_pred = np.exp(cat_boost.predict(X_test))
print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")

Точность модели по метрике MAPE: 10.94%

In [144]:
y_pred = np.exp(cat_boost.predict(X_valid))
print(f"Точность модели по метрике MAPE: {(mape(y_valid, y_pred))*100:0.2f}%")

Точность модели по метрике MAPE: 10.86%

In [145]:
y_pred = np.exp(cat_boost.predict(X_CatB_val))
# округлим итоговую цену
y_pred_round = np.round(y_pred/100000,2)*100000
sample_submission['price'] = y_pred_round
sample_submission.to_csv('submission_1.csv', index=False)

sabmission 19.93440%

In [146]:
# # Попробуем предсказать по бренду
# 
# X_CatB_brand_test = X_CatB_test
# X_CatB_brand_val = X_CatB_val
# 
# cat_features = summary_cat
# cat_features.remove('brand')
# for i in X_CatB_brand_test['brand'].unique():
# 
#     # разбиваем 70 на 30
#     X_brand = X_CatB_brand_test[X_CatB_brand_test.brand == i].drop('brand',axis = 1)
#     
#     y_brand = y[X_CatB_brand_test[X_CatB_brand_test.brand == i].index]
#     
#     X_train, X_test, y_train, y_test = train_test_split(
#         X_brand,
#         y_brand,
#         test_size=0.30,
#         random_state=42)
#     
#     # создаем pool
#     feature_names = list(X_brand.columns)
#     
#     pool_brand = Pool(X_brand, np.log(y_brand), feature_names = feature_names, cat_features = cat_features)
#     
#     # модель на 10 000 итераций learning_rate=0.1
#     cat_boost = CatBoostRegressor(iterations=10000,
#                               random_seed=42,
#                               learning_rate=0.1,
#                               eval_metric='MAPE',
#                               od_type='Iter',
#                               od_wait=200,
#                               silent=True)
#     cat_boost.fit(
#         X=pool_brand,
#         eval_set=(X_test, np.log(y_test)),
#         verbose_eval=0,
#         use_best_model=True,
#         plot=True)
#     
#     y_pred = np.exp(cat_boost.predict(X_test))
#     print(i, f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
#     
#     X_brand_val = X_CatB_brand_val[X_CatB_brand_val.brand == i].drop('brand',axis = 1)
#     
#     y_pred_val = np.exp(cat_boost.predict(X_brand_val))
#     
#     sample_submission.loc[X_CatB_brand_val[X_CatB_brand_val.brand == i].index, 'price'] = y_pred_val
#     
# sample_submission

In [147]:
# sample_submission['price'] = sample_submission['price'].apply(lambda x: x - x*0.4)

In [148]:
# sample_submission.to_csv('submission_1.csv', index=False)

sabmission 21.93695%

## Random Forest Regressor

In [149]:
# подготовим данные

# Преобразуем бинарные признаки с помощью LabelEncoder

X_rf_cat_bin = data[bin_list].copy()

le = LabelEncoder()
for i in X_rf_cat_bin:
    X_rf_cat_bin.loc[X_rf_cat_bin.index,i] = le.fit_transform(X_rf_cat_bin.loc[X_rf_cat_bin.index,i])
    
X_rf_cat_bin = X_rf_cat_bin.values
    
print('X_rf_cat_bin ',X_rf_cat_bin.shape)

# числовые признаки

sc = StandardScaler()

X_rf_cat_num = data.select_dtypes(include=['int','float']).drop(['sell_id','price'],axis=1)
X_rf_cat_num = sc.fit_transform(X_rf_cat_num.values)

print('X_rf_cat_num ',X_rf_cat_num.shape)

# dummy 

X_rf_cat_dumm = X_CatB_dumm.values

print('X_rf_cat_dumm ',X_rf_cat_dumm.shape)

# Категориальные

X_rf_cat = data.select_dtypes(include='object')
X_rf_cat

X_rf_cat = OneHotEncoder(sparse = False).fit_transform(X_rf_cat.drop('equipment_list',axis=1).values)

print('X_rf_cat ',X_rf_cat.shape)

# объеденим

X_rf_full = np.hstack([X_rf_cat_bin,X_rf_cat_num,X_rf_cat_dumm,X_rf_cat])

print('X_rf_full ',X_rf_full.shape)

# разделим на тест и валидацию

X_rf_test = X_rf_full[data.query('price != 0').index]
X_rf_val = X_rf_full[data.query('price == 0').index]

print('X_rf_test ',X_rf_test.shape)
print('X_rf_val ',X_rf_val.shape)

In [150]:
# подготовим модель, разобьем на три фолда

valid_frac = 0.15
test_frac = 0.15

valid_count = int(valid_frac * X.shape[0])
test_count = int(test_frac * X.shape[0])

X_train, X_valid_test, y_train, y_valid_test = train_test_split(X_rf_test, y,
    test_size=(valid_count + test_count), random_state=42)

X_valid, X_test, y_valid, y_test = train_test_split(
    X_valid_test, y_valid_test, test_size=test_count, random_state=42)

In [151]:
# # модель на базовых настройках
# rf_log = RandomForestRegressor(random_state=42, n_jobs=-1, verbose=1)
# rf_log.fit(X_train, np.log(y_train))
# 
# 
# y_pred = np.exp(rf_log.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
# y_pred = np.exp(rf_log.predict(X_valid))
# print(f"Точность модели по метрике MAPE: {(mape(y_valid, y_pred))*100:0.2f}%")

Точность модели по метрике MAPE: 14.96%

Точность модели по метрике MAPE: 14.73%

In [152]:
# y_pred = np.exp(rf_log.predict(X_rf_val))
# sample_submission['price'] = y_pred
# sample_submission.to_csv('submission_1.csv', index=False)

submission 41.76971

## ExtraTreesRegressor

In [153]:
# # # модель на базовых настройках
# 
# extrees = ExtraTreesRegressor(random_state=42, n_jobs=-1, verbose=1)
# 
# extrees.fit(X_train, np.log(y_train))
# 
# y_pred = np.exp(extrees.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
# 
# y_pred = np.exp(extrees.predict(X_valid))
# print(f"Точность модели по метрике MAPE: {(mape(y_valid, y_pred))*100:0.2f}%")

Точность модели по метрике MAPE: 15.54%
Точность модели по метрике MAPE: 15.20%

In [154]:
# y_pred = np.exp(extrees.predict(X_rf_val))
# sample_submission['price'] = y_pred
# sample_submission.to_csv('submission_1.csv', index=False)

submission 79.60793%

## AdaBoostRegressor

In [155]:
# # модель на базовых настройках
# 
# abr = AdaBoostRegressor(random_state=42)
# 
# abr.fit(X_train, np.log(y_train))
# 
# y_pred = np.exp(abr.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
# 
# y_pred = np.exp(abr.predict(X_valid))
# print(f"Точность модели по метрике MAPE: {(mape(y_valid, y_pred))*100:0.2f}%")

In [156]:
# y_pred = np.exp(abr.predict(X_rf_val))
# sample_submission['price'] = y_pred
# sample_submission.to_csv('submission_1.csv', index=False)

submission 68.59933%

## XGBRegressor

In [157]:
# # модель на базовых настройках
# 
# xg_reg = xgb.XGBRegressor(
#      random_state=42, 
#      n_jobs=-1,
# )
# 
# xg_reg.fit(X_train,np.log(y_train))
# 
# y_pred = np.exp(xg_reg.predict(X_test))
# print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred))*100:0.2f}%")
# 
# y_pred = np.exp(xg_reg.predict(X_valid))
# print(f"Точность модели по метрике MAPE: {(mape(y_valid, y_pred))*100:0.2f}%")

In [158]:
# y_pred = np.exp(xg_reg.predict(X_rf_val))
# sample_submission['price'] = y_pred
# sample_submission.to_csv('submission_1.csv', index=False)

submission 41.84023%

## Stacking

In [159]:
# X_train, X_test, y_train, y_test = train_test_split(X_rf_test, y.values, random_state=42, test_size=0.3)
# 
# estimators = [
#     ('xgb', xgb.XGBRegressor(random_state=42, n_jobs=-1)),
#     ('rfr', RandomForestRegressor(random_state=42, n_jobs=-1, verbose=1))]
# sr_log = StackingRegressor(
#     estimators = estimators,
#     final_estimator = LinearRegression()
# )
# 
# sr_log.fit(X_train, np.log(y_train))
# 
# 
# y_pred = np.exp(sr_log.predict(X_test))
# 
# print(f"Точность модели по метрике MAPE: {(mape(y_test, y_pred) * 100):0.2f}%.")

In [160]:
# y_pred = np.exp(sr_log.predict(X_rf_val))
# sample_submission['price'] = y_pred
# sample_submission.to_csv('submission_1.csv', index=False)

submission 41.30537%